# N-Gramを利用して変数名の提案

ライブラリーと関数の宣言

In [4]:
def match_graphs(graph, graphs):
  for search in graphs:
    if graph.graph == search.graph:
      return search
  return None

In [9]:
def combine_node(raw_doc):
  data = []
  sub_data = ''
  for i in range(len(raw_doc)): 
    sub_data = raw_doc[i][0]
    for j in range(1, len(raw_doc[i])):
      sub_data = sub_data + ' X ' + raw_doc[i][j]
    data.append(sub_data)
  return data

In [10]:
from sklearn.metrics.pairwise import cosine_similarity
def similar(test_vec, test_dict:dict, graphs, topn = 10):
  dists = []
  index_to_key = []
  i = 0
  result = []
  for key in test_dict.keys():
    dist = cosine_similarity(test_vec.reshape(1, -1), test_dict[key].reshape(1, -1))
    index_to_key.append(key)
    dists.append(float(dist))


  index_to_key = np.array(index_to_key)
  dists = np.array(dists) 
  best = dists.argsort()[::-1] 
  for sim in best:
    try:
      result.append([index_to_key[sim], graphs[sim].graph['label1'], graphs[sim].graph['label2'], float(dists[sim])])
    except KeyError:
      print(sim)
      continue
  return result[:topn]

In [11]:
def sim_combine(sim_list):
  result = {}
  result_count ={}
  sim = 0
  for i in range(len(sim_list)):
    try:
      if sim_list[i][2] in result[sim_list[i][1]].keys():
        continue
    except KeyError:
      result[sim_list[i][1]] = {}
      result_count[sim_list[i][1]] = {}
    temp = sim_list[i]
    sum = 0
    count = 0
    for j in range(i,len(sim_list)):
      if temp[2] == sim_list[j][2] and temp[1] == sim_list[j][1]:
        sum = sum + sim_list[j][3]
        count += 1
    avg = sum/count
    result[sim_list[i][1]].update({sim_list[i][2] : avg})
    result_count[sim_list[i][1]].update({sim_list[i][2] : count})
  return result, result_count

In [5]:
from MyWeisfeilerLehaman import WeisfeilerLehmanHashing
from gensim.models.doc2vec import TaggedDocument
import numpy as np

def WLchange(graphs, wl_iterations, attributed, erase_base_features):
    documents = [
    WeisfeilerLehmanHashing(
        graph, wl_iterations, attributed, erase_base_features
    )
    for graph in graphs
    ]

    documents = [
        doc.get_graph_features() for i, doc in enumerate(documents)
    ]
    return documents

訓練ファイルの入力

In [6]:
import glob
import networkx as nx
import matplotlib.pyplot as plt

loopfile = glob.glob('/Users/wintor7/Research/loopset/*.gpickle', recursive=True)
graphfile = glob.glob('/Users/wintor7/Research/graphset_new/*.gpickle', recursive=True)

graphs = []
for i in range(int(len(graphfile)/10)):
  G = nx.read_gpickle(graphfile[i])
  graphs.append(G)
loop_graphs = []
for i in range(int(len(loopfile)/10)):
  G = nx.read_gpickle(loopfile[i])
  loop_graphs.append(G)

graphfile = glob.glob("/Users/wintor7/report7/graph/var/*.gpickle")
loopfile = glob.glob("/Users/wintor7/report7/graph/loop/*.gpickle")



for gfile in graphfile:
  G = nx.read_gpickle(gfile)
  graphs.append(G)


for i in loopfile:
  G = nx.read_gpickle(i)
  loop_graphs.append(G)

len(graphs)


9993

抽出範囲拡大部分の追加

In [7]:
documents = WLchange(graphs, 2, True, False)
for i in range(len(graphs)):
    loop = match_graphs(graphs[i], loop_graphs)
    documents[i].append('loop')
    if loop:
        loop = WeisfeilerLehmanHashing(loop, 2, True, False)
        loop = loop.get_graph_features()
        for j in range(len(loop)):
            documents[i].append(loop[j])
    else:
        documents[i].append('Null')

In [8]:
doc_withtag = [
    TaggedDocument(words=doc, tags=[str(i)])
    for i, doc in enumerate(documents)
]

raw_doc = []
for i in range(len(doc_withtag)):
  raw_doc.append(doc_withtag[i].words)

In [12]:
data = combine_node(raw_doc)


In [13]:
data[1]

'29 X 29 49 X 29 49 49 51 57 59 66 X 49 X 49 51 57 59 66 X 49 51 57 59 66 51 57 57 None 59 59 75 66 75 X 59 X 59 59 75 X 59 59 75 59 66 75 75 None X 59 X 59 66 75 X 59 66 75 66 75 75 None X 66 X 66 75 X 66 75 75 None X 75 X 75 None X 75 None None X 75 X 75 None X 75 None None X 75 X 75 None X 75 None None X 51 X 51 57 X 51 57 57 None X 57 X 57 None X 57 None None X 66 X 66 75 X 66 75 75 None X 75 X 75 None X 75 None None X 57 X 57 None X 57 None None X loop X Null'

モデルの準備

In [14]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

token_pattern = r'\b\w+\b'


In [15]:
ngram=2
vectorizer = CountVectorizer(ngram_range=(1,ngram), token_pattern=token_pattern, min_df=1)

X = vectorizer.fit(data)

vocab_2 = vectorizer.get_feature_names()

/Users/wintor7/miniconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


抽象構文木の抽出

In [16]:
from Mygraph2vec import Graph2Vec
import networkx as nx
from ast_draw import CodeTransformer, counter
import glob
import ast
import node_to_vec as n2v

テストファイルの入力

In [283]:
import glob

path = '/Users/wintor7/report7/test_file'
file = glob.glob(path + '/**_search_words_**.py')
func = 'search_words'
file

['/Users/wintor7/report7/test_file/e205760_search_words_sub_set.py',
 '/Users/wintor7/report7/test_file/e215701_search_words_intermediate_result.py',
 '/Users/wintor7/report7/test_file/e205715_search_words_searched.py',
 '/Users/wintor7/report7/test_file/e195760_search_words_search_word.py',
 '/Users/wintor7/report7/test_file/e195743_search_words_one_youso_list.py',
 '/Users/wintor7/report7/test_file/e215710_search_words_words_dict.py',
 '/Users/wintor7/report7/test_file/e205708_search_words_save_files.py',
 '/Users/wintor7/report7/test_file/e195735_search_words_result.py',
 '/Users/wintor7/report7/test_file/e215703_search_words_count.py',
 '/Users/wintor7/report7/test_file/e195730_search_words_file_list.py',
 '/Users/wintor7/report7/test_file/e195763_search_words_result_files.py',
 '/Users/wintor7/report7/test_file/e205728_search_words_items_.py',
 '/Users/wintor7/report7/test_file/e215716_search_words_count.py',
 '/Users/wintor7/report7/test_file/e205717_search_words_words.py',
 '/Us

In [520]:
file[-3]

'/Users/wintor7/report7/test_file/e205717_search_words_words.py'

In [19]:
var, for_var, func_var, func_params, func_return = {}, {}, {}, {}, {}
dict_func, var_counter = {},{}
node_to_id = {}
id_to_node = {}
node_to_id, id_to_node = n2v.create_node_lib()      #create node corpus
var_to_id, func_to_id = {},{}
id_to_var, id_to_func = {},{}

In [573]:
test_var = 'test_var1'
with open(file[-3], 'r') as fh:
  content = fh.read()

In [574]:
r_node = ast.parse(content)

transformer = CodeTransformer()
r_node = transformer.visit(r_node)

In [575]:
dict_func = counter(dict_func, transformer.var_in_func)

name_line = transformer.name_line
node_list = {}

for key in name_line.keys():
  var_node = {}
  for var, value in name_line[key].items():
    for i in value:
      transformer.track_line(r_node, i)
      if var not in var_node.keys():
        var_node[var] = [transformer.target_var]
      elif transformer.target_var not in var_node[var]:
        var_node[var].append(transformer.target_var)
  node_list[key] = var_node
node_list

{'make_index': {'index': [<_ast.Assign at 0x130596760>,
  'sentence': [<_ast.Assign at 0x130560b50>,
  'filenames': [<_ast.For at 0x1305607f0>, <_ast.Expr at 0x1304b4fa0>],
  'fh': [<_ast.With at 0x130748250>, <_ast.Expr at 0x1307489a0>],
  'names': [<_ast.Assign at 0x1304b4340>,
   <_ast.Assign at 0x1304b45b0>]},
 'search_words': {'test_var': [<_ast.Assign at 0x1307f83a0>,
  'index': [<_ast.Assign at 0x1307f83a0>, <_ast.Assign at 0x1307f8700>],
  'test_var1': [<_ast.Assign at 0x1307f83a0>, <_ast.For at 0x1307f8670>]}}

In [576]:
loop_list = {}        #
for key in name_line.keys():
  loop_temp_list = {}
  for var, value in name_line[key].items():
    for i in value:
      transformer.track_loop(r_node,i)
      if var not in loop_temp_list.keys():
        loop_temp_list[var] = [transformer.target_loop]
      elif transformer.target_loop not in loop_temp_list[var]:
        loop_temp_list[var].append(transformer.target_loop)
  loop_list[key] = loop_temp_list 
loop_list

{'make_index': {'index': [None, <_ast.For at 0x1307489d0>],
  'sentence': [<_ast.For at 0x1307489d0>, <_ast.For at 0x1305607f0>],
  'filenames': [<_ast.For at 0x1305607f0>, <_ast.For at 0x1307489d0>],
  'fh': [<_ast.For at 0x1305607f0>],
  'names': [<_ast.For at 0x1307489d0>]},
 'search_words': {'test_var': [<_ast.Try at 0x1307f82e0>],
  'index': [<_ast.Try at 0x1307f82e0>],
  'test_var1': [<_ast.Try at 0x1307f82e0>]}}

In [577]:
func_list = list(transformer.var_in_func.keys())
var_list = []
for i,v in enumerate(transformer.var_in_func):
  for i in transformer.var_in_func[v]:
    var_list.append(i)
var_to_id, id_to_var = n2v.name_corpus(var_list, var_to_id, id_to_var)
func_to_id, id_to_func = n2v.name_corpus(func_list, func_to_id, id_to_func)

In [578]:
graph_list = []
graph = nx.DiGraph()
str_to_node = {}
for var_node in node_list[func][test_var]:
  root = len(str_to_node)
  n2v.create_graph(var_node, node_to_id, graph, str_to_node, root)
  for node in graph.nodes():
    graph.nodes[node]['feature'] = str_to_node[node]
  root += 1  
graph_list.append(graph) 

In [579]:
graphs_loop = []
graph = nx.DiGraph()
str_to_node = {}
for var_node in loop_list[func][test_var]:
  if var_node == None:
    continue
  root = len(str_to_node)
  n2v.create_graph(var_node, node_to_id, graph, str_to_node, root)
  for node in graph.nodes():
    graph.nodes[node]['feature'] = str_to_node[node]
  root += 1  
graphs_loop.append(graph) 

WL再ラベル操作

In [580]:
documents = [
  WeisfeilerLehmanHashing(
      graph, 2, True, False)
  for graph in graph_list
]


documents_loop = [
  WeisfeilerLehmanHashing(
      graph, 2, True, False)
  for graph in graphs_loop
]

In [581]:
documents = [
  doc.get_graph_features() for i, doc in enumerate(documents)
]

documents_loop = [
  doc.get_graph_features() for i, doc in enumerate(documents_loop)
]

In [582]:
documents[0].append('loop')
if documents_loop != None:
  for j in range(len(documents_loop[0])):
    documents[0].append(documents_loop[0][j])
else:
  documents[0].append('Null')

In [583]:
test_data = combine_node(documents)
type(test_data)

list

In [584]:
test_data

['13 X 13 49 66 X 13 49 66 49 64 66 66 76 X 66 X 66 76 X 66 76 76 None X 76 X 76 None X 76 None None X 49 X 49 64 66 X 49 64 66 64 63 66 75 66 75 X 66 X 66 75 X 66 75 75 None X 75 X 75 None X 75 None None X 64 X 64 63 66 75 X 64 63 66 75 63 64 66 75 75 None X 66 X 66 75 X 66 75 75 None X 75 X 75 None X 75 None None X 63 X 63 64 X 63 64 64 63 66 75 X 64 X 64 63 66 75 X 64 63 66 75 63 57 66 75 75 None X 66 X 66 75 X 66 75 75 None X 75 X 75 None X 75 None None X 63 X 63 57 X 63 57 57 None X 57 X 57 None X 57 None None X 75 X 75 None X 75 None None X 75 X 75 None X 75 None None X 16 X 16 13 66 66 X 16 13 66 66 13 35 66 66 75 66 76 X 66 X 66 76 X 66 76 76 None X 76 X 76 None X 76 None None X 66 X 66 75 X 66 75 75 None X 75 X 75 None X 75 None None X 13 X 13 35 66 X 13 35 66 35 49 66 82 66 76 X 66 X 66 76 X 66 76 76 None X 76 X 76 None X 76 None None X 35 X 35 49 66 82 X 35 49 66 82 49 64 66 66 75 82 None X 66 X 66 75 X 66 75 75 None X 75 X 75 None X 75 None None X 82 X 82 None X 82 None Non

コーパスの準備

In [585]:
ngram=2

vectorizer = CountVectorizer(ngram_range=(1,ngram), token_pattern=token_pattern, min_df=1)
test_vec = vectorizer.fit_transform(test_data)
vocab_test = vectorizer.get_feature_names_out()
test_vectors = test_vec.toarray()
df = pd.DataFrame(test_vectors, columns=vocab_test)
df

,108,108 49,108 none,108 x,11,11 11,11 48,11 49,11 57,11 68,...,x 49,x 57,x 63,x 64,x 66,x 68,x 75,x 76,x 82,x loop
0,7,3,3,1,21,3,3,3,6,3,...,21,15,18,18,87,3,90,18,6,1


In [586]:
for i in range(len(vocab_test)):
  if vocab_test[i] not in vocab_2:
    print(i)
    vocab_2.append(vocab_test[i])

モデルの訓練

In [587]:
train_vec = {}
ngram=2
vectorizer = CountVectorizer(ngram_range=(1,ngram), token_pattern=token_pattern, min_df=1 , vocabulary= vocab_2)
X = vectorizer.fit_transform(data)
raw_vectors = X.toarray()
#df = pd.DataFrame(raw_vectors, columns=vocab_2)
for i in range(len(raw_vectors)):
  train_vec[doc_withtag[i].tags[0]] = raw_vectors[i]

In [588]:
test_vec = vectorizer.fit_transform(test_data)
vocab_test = vectorizer.get_feature_names_out()
test_vectors = test_vec.toarray()
test_vectors = test_vectors[0]

In [589]:

cosine_similarity(train_vec['0'].reshape(1, -1),test_vectors.reshape(1, -1))


array([[0.91514691]])

In [590]:
test_vectors.shape

(2092,)

類似度の計算

In [591]:
result = similar(test_vectors,train_vec,graphs,topn=100)

N-gramにより提案結果

In [592]:
result, result_count = sim_combine(result)
print(result)
print(result_count)

{'search_words': {'keywords_for_search': 0.9989986027312507, 'names': 0.9975165214738414, 'search': 0.9946123750485458, 'index': 0.9928625724241522, 'result': 0.9936409412552768, 'words': 0.9934546389646055, 'keywords': 0.995183190065394, 'file_list': 0.9930008192434482, 'search_word_list': 0.9943278611479192, 'for_file': 0.9936352050983485, 'filename': 0.9931566876163171, 'word': 0.992617261024483, 'result_file': 0.9925970338710742, 'search_result': 0.9923168293807022, 'in_file': 0.9922135057783251, 'search_list': 0.9918969568527943}, 'dfs': {'max_size': 0.995646003900327}, 'check_location_is_safe': {'arr': 0.9928592809599277}, 'slash': {'var': 0.9926192472897601}, 'make_index': {'filename': 0.9924209461182144}}
{'search_words': {'keywords_for_search': 7, 'names': 6, 'search': 13, 'index': 17, 'result': 5, 'words': 8, 'keywords': 1, 'file_list': 11, 'search_word_list': 1, 'for_file': 7, 'filename': 1, 'word': 3, 'result_file': 7, 'search_result': 3, 'in_file': 3, 'search_list': 3}, 'd

課題文とのマッチング

In [594]:
import get_alternateName as ga

マッチングにより提案結果

In [595]:
all_list = ga.get_altName(result[func])
all_list

['element']

In [596]:
len(all_list)

1